In [1]:
from cs231n.data_utils import get_CIFAR10_data

# load cifar10 data
data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [2]:
# split and normalize data
X_val, y_val = data['X_val'], data['y_val']
X_train, y_train = data['X_train'], data['y_train']
X_test, y_test = data['X_test'], data['y_test']

# X_train /= 255.
# X_val /= 255.
# X_test /= 255.

In [3]:
import keras
from keras.utils import to_categorical

# 1 hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)

Using TensorFlow backend.


In [21]:
# build network
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential

import numpy as np

model = Sequential()

def my_init(shape, dtype=None):
    weight_scale = 1e-3
    return weight_scale * np.random.randn(*shape).astype(dtype)

keras.backend.set_image_data_format('channels_first')
conv_kwargs = {'filters': 32, 'kernel_size': 3, 
               'padding': 'same', 'kernel_initializer': my_init}
batch_kwargs = {'momentum': 0.9,
                'epsilon': 1e-5, 'moving_variance_initializer': 'zeros'}

shape = X_train.shape[1:]
model.add(Conv2D(input_shape=shape, **conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(MaxPool2D())

model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(Conv2D(**conv_kwargs))
model.add(BatchNormalization(axis=1, **batch_kwargs))
model.add(Activation('relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(500, kernel_initializer=my_init))
model.add(BatchNormalization(**batch_kwargs))
model.add(Activation('relu'))
model.add(Dense(10, kernel_initializer=my_init))
model.add(Activation('softmax'))

In [22]:
# set optimizer
from keras.optimizers import Adam

optim = Adam(decay=5e-3)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
# train model
history = model.fit(X_train, y_train,
                    batch_size=100, epochs=10,
                    validation_data=(X_val, y_val),
                    shuffle=True)

Train on 49000 samples, validate on 1000 samples
Epoch 1/10
49000/49000 [==============================] - 179s - loss: 1.2357 - acc: 0.5528 - val_loss: 0.9443 - val_acc: 0.6600
Epoch 2/10
49000/49000 [==============================] - 177s - loss: 0.7886 - acc: 0.7225 - val_loss: 0.7566 - val_acc: 0.7410
Epoch 3/10
49000/49000 [==============================] - 177s - loss: 0.6106 - acc: 0.7872 - val_loss: 0.6373 - val_acc: 0.7770
Epoch 4/10
49000/49000 [==============================] - 177s - loss: 0.4816 - acc: 0.8329 - val_loss: 0.6384 - val_acc: 0.7860
Epoch 5/10
49000/49000 [==============================] - 177s - loss: 0.3747 - acc: 0.8716 - val_loss: 0.6196 - val_acc: 0.7810
Epoch 6/10
49000/49000 [==============================] - 177s - loss: 0.2725 - acc: 0.9097 - val_loss: 0.6220 - val_acc: 0.8040
Epoch 7/10
49000/49000 [==============================] - 177s - loss: 0.1894 - acc: 0.9389 - val_loss: 0.6751 - val_acc: 0.7820
Epoch 8/10
49000/49000 [========================

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=100)

In [ ]:
test_acc = np.mean(y_pred == y_test)
print 'Test Accuracy: {}'.format(test_acc)